# PoSyg Economic Model Analysis
Token distribution, rewards, and stability mechanisms

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import optimize
import json
from pathlib import Path

## 1. Token Distribution Evolution

In [ ]:
# Analyze wealth concentration over time
def analyze_token_distribution(metrics_df):
    # Calculate concentration metrics
    gini_evolution = metrics_df['gini_coefficient']
    
    # Calculate Theil index approximation
    theil_index = []
    for _, row in metrics_df.iterrows():
        # Approximate from Gini
        g = row['gini_coefficient']
        theil = -np.log(1 - g) if g < 1 else np.inf
        theil_index.append(theil)
    
    # HHI (Herfindahl-Hirschman Index) approximation
    hhi = gini_evolution ** 2 * 10000  # Scaled approximation
    
    return pd.DataFrame({
        'epoch': metrics_df['epoch'],
        'gini': gini_evolution,
        'theil': theil_index,
        'hhi': hhi
    })

## 2. Reward Optimization Analysis

In [ ]:
# Model optimal reward distribution
def optimize_reward_distribution(validators_df, target_gini=0.4):
    # Current state
    stakes = validators_df['final_stake'].values
    activities = validators_df['blocks_proposed'].values + validators_df['blocks_attested'].values * 0.1
    governance = validators_df['governance_votes'].values
    
    # Normalize
    stakes_norm = stakes / stakes.sum()
    activities_norm = activities / max(activities.sum(), 1)
    governance_norm = governance / max(governance.sum(), 1)
    
    # Optimization objective: minimize Gini while maintaining security
    def objective(weights):
        w_stake, w_activity, w_governance = weights
        scores = w_stake * stakes_norm + w_activity * activities_norm + w_governance * governance_norm
        
        # Calculate Gini of resulting distribution
        sorted_scores = np.sort(scores)
        n = len(sorted_scores)
        cumsum = np.cumsum(sorted_scores)
        gini = (2 * np.sum((np.arange(1, n+1) * sorted_scores))) / (n * cumsum[-1]) - (n + 1) / n
        
        # Penalty for deviating from target
        return (gini - target_gini) ** 2
    
    # Constraints: weights sum to 1, all positive
    constraints = [
        {'type': 'eq', 'fun': lambda w: w.sum() - 1},
        {'type': 'ineq', 'fun': lambda w: w[0] - 0.2}  # Min stake weight for security
    ]
    
    # Optimize
    result = optimize.minimize(
        objective, 
        x0=[0.4, 0.4, 0.2], 
        bounds=[(0.2, 0.8), (0.1, 0.7), (0.1, 0.5)],
        constraints=constraints
    )
    
    return {
        'optimal_weights': result.x,
        'achieved_gini': np.sqrt(result.fun) + target_gini,
        'success': result.success
    }

## 3. Slashing Economics

In [ ]:
# Analyze slashing impact on network economics
def analyze_slashing_economics(validators_df, metrics_df):
    # Group validators by slashing count
    slash_groups = validators_df.groupby('slashing_count').agg({
        'id': 'count',
        'final_stake': ['mean', 'sum'],
        'final_synergy_score': 'mean'
    })
    
    # Calculate economic impact
    total_initial_stake = 1000000  # Assuming 1M total initial stake
    total_final_stake = validators_df['final_stake'].sum()
    stake_burned = total_initial_stake - total_final_stake
    
    # Slashing effectiveness
    slashed_validators = validators_df[validators_df['slashing_count'] > 0]
    if len(slashed_validators) > 0:
        avg_penalty_per_slash = (
            (total_initial_stake / len(validators_df) - slashed_validators['final_stake'].mean()) / 
            slashed_validators['slashing_count'].mean()
        )
    else:
        avg_penalty_per_slash = 0
    
    return {
        'total_stake_burned': stake_burned,
        'burn_rate': stake_burned / total_initial_stake,
        'avg_penalty_per_slash': avg_penalty_per_slash,
        'slashed_validator_ratio': len(slashed_validators) / len(validators_df),
        'slash_groups': slash_groups
    }

## 4. Stability Fund Modeling

In [ ]:
# Model algorithmic stability fund behavior
def model_stability_fund(market_volatility=0.3, fund_size_ratio=0.15, simulation_periods=365):
    np.random.seed(42)
    
    # Initialize
    peg_price = 1.0
    market_price = peg_price
    fund_size = fund_size_ratio * 1000000  # 15% of total supply
    intervention_threshold = 0.05  # 5% deviation
    
    history = []
    
    for t in range(simulation_periods):
        # Market shock (random walk with volatility)
        shock = np.random.normal(0, market_volatility / np.sqrt(365))
        market_price *= (1 + shock)
        
        # Calculate deviation
        deviation = abs(market_price - peg_price) / peg_price
        
        # Fund intervention
        intervention_amount = 0
        if deviation > intervention_threshold:
            # Intervention strength proportional to deviation
            intervention_strength = min(deviation * 2, 0.5)
            intervention_amount = fund_size * intervention_strength
            
            # Apply intervention (simplified)
            if market_price > peg_price:
                # Sell to reduce price
                market_price *= (1 - intervention_strength * 0.1)
            else:
                # Buy to increase price
                market_price *= (1 + intervention_strength * 0.1)
            
            # Reduce fund size
            fund_size -= intervention_amount * 0.01  # Cost of intervention
        
        history.append({
            'period': t,
            'market_price': market_price,
            'deviation': deviation,
            'fund_size': fund_size,
            'intervention': intervention_amount
        })
    
    return pd.DataFrame(history)

## 5. Inflation and Supply Analysis

In [ ]:
# Model token supply and inflation
def analyze_inflation_impact(initial_supply=1000000, inflation_rate=0.05, epochs=1000):
    supply_history = [initial_supply]
    
    for epoch in range(1, epochs):
        # Annual inflation converted to per-epoch
        epoch_inflation = inflation_rate / 365
        new_tokens = supply_history[-1] * epoch_inflation
        supply_history.append(supply_history[-1] + new_tokens)
    
    # Calculate metrics
    inflation_df = pd.DataFrame({
        'epoch': range(epochs),
        'total_supply': supply_history,
        'epoch_inflation': np.diff([initial_supply] + supply_history),
        'cumulative_inflation': (np.array(supply_history) - initial_supply) / initial_supply
    })
    
    return inflation_df

## 6. Economic Equilibrium Analysis

In [ ]:
# Find economic equilibrium points
def find_economic_equilibrium(validators_df, target_decentralization=0.6):
    # Current state
    gini = validators_df['final_stake'].sort_values()
    n = len(gini)
    cumsum = np.cumsum(gini)
    current_gini = (2 * np.sum((np.arange(1, n+1) * gini))) / (n * cumsum[-1]) - (n + 1) / n
    
    # Target Gini for decentralization
    target_gini = 1 - target_decentralization
    
    # Calculate required redistribution
    if current_gini > target_gini:
        # Need to redistribute from top to bottom
        top_10_pct = validators_df.nlargest(int(n * 0.1), 'final_stake')
        bottom_50_pct = validators_df.nsmallest(int(n * 0.5), 'final_stake')
        
        redistribution_needed = (
            (current_gini - target_gini) * 
            validators_df['final_stake'].sum()
        )
        
        return {
            'current_gini': current_gini,
            'target_gini': target_gini,
            'redistribution_needed': redistribution_needed,
            'top_10_stake_ratio': top_10_pct['final_stake'].sum() / validators_df['final_stake'].sum(),
            'bottom_50_stake_ratio': bottom_50_pct['final_stake'].sum() / validators_df['final_stake'].sum()
        }
    else:
        return {
            'current_gini': current_gini,
            'target_gini': target_gini,
            'status': 'Already at or below target decentralization'
        }

## 7. Comprehensive Economic Report

In [ ]:
# Generate economic analysis report
scenarios = ['baseline', 'cartel_attack', 'high_byzantine']
economic_results = {}

for scenario in scenarios:
    try:
        path = f"results_{scenario}"
        metrics = pd.read_csv(f"{path}/metrics_history.csv")
        validators = pd.read_csv(f"{path}/final_validator_states.csv")
        
        # Run all analyses
        economic_results[scenario] = {
            'distribution': analyze_token_distribution(metrics),
            'optimal_weights': optimize_reward_distribution(validators),
            'slashing': analyze_slashing_economics(validators, metrics),
            'equilibrium': find_economic_equilibrium(validators)
        }
    except Exception as e:
        print(f"Error analyzing {scenario}: {e}")

# Visualize key economic metrics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Economic Analysis Summary', fontsize=16)

# Implementation of visualization code
plt.tight_layout()
plt.show()